In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().resolve().parents[0]  # repo root if cwd==repo/notebooks
sys.path.insert(0, str(ROOT / "src"))



In [2]:
from openai import OpenAI
from qdrant_client import QdrantClient
from tracerag.rag import (
    QdrantRetrievalConfig,
    build_openai_embedding_fn,
    build_openai_chat_fn,
    build_qdrant_retriever,
    naive_rag,
)

client = OpenAI()
qdrant = QdrantClient(host="localhost", port=6333)

embed_fn = build_openai_embedding_fn(client, model="text-embedding-3-small")
chat_fn = build_openai_chat_fn(client, model="gpt-4.1-mini")

retriever = build_qdrant_retriever(
    qdrant=qdrant,
    embedding_fn=embed_fn,
    config=QdrantRetrievalConfig(collection_name="catalog_embeddings"),
)

resp = naive_rag(
    user_query="Generate telemetry configuration for Cisco IOS XR about BGP",
    retriever=retriever,
    chat_fn=chat_fn,
    system_prompt="You're a network configuration assistant. Use the provided context to answer the user's query accurately.",
    top_k=10,
)

print(resp.answer)

sensor-group Cisco-IOS-XR-BGP {
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp-state;
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp-attributes;
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp-rib;
}

subscription BGP-Telemetry-Subscription {
  sensor-group-id Cisco-IOS-XR-BGP;
  sample-interval 10000;
  encoding self-describing-gpb;
  transport udp {
    udp {
      local-port 57500;
      remote-ip 192.0.2.1;
      remote-port 57500;
    }
  }
}
